In [35]:
from huggingface_hub import login

#허깅페이스 액세스토큰
hf_token = ""

# 허깅페이스 로그인
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/baekkwsl/.cache/huggingface/token
Login successful


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = torch.bfloat16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-09-29 13:58:06.430792: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-29 13:58:06.438682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 13:58:06.446389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 13:58:06.448655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 13:58:06.454642: I tensorflow/core/platform/cpu_feature_guar

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.9: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [3]:
## Peft 모델(fine-tunning)
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj","embed_tokens",],
                      # "embed_tokens", "lm_head",]
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3103,
    use_rslora = True,
    loftq_config = None,
)
model.print_trainable_parameters()

Unsloth: Offloading input_embeddings to disk to save VRAM


/home/baekkwsl/anaconda3/envs/gmlb/lib/python3.10/site-packages/unsloth/models/_utils.py:866: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filename

Unsloth: Casting embed_tokens to float32


# 데이터셋 정리

In [5]:
from datasets import load_dataset, concatenate_datasets

alpaca_dataset = load_dataset("beomi/KoAlpaca-v1.1a",  split = "train")
kin_dataset = load_dataset("seungjune/RBC_kin_data",  split = "train")
dossa_dataset = load_dataset("seungjune/RBC_dossa_data",  split = "train")

print(alpaca_dataset)
print(kin_dataset)
print(dossa_dataset)

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 21155
})
Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 40925
})
Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 61924
})


In [6]:
dataset = concatenate_datasets([alpaca_dataset, kin_dataset, dossa_dataset])
print(dataset)

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 124004
})


In [7]:
qa_prompt = """다음은 작업을 설명하는 명령입니다. 요청을 적절하게 완료하는 자세한 응답을 작성하세요.

### 지침:
{}

### 응답:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    texts = []
    for instruction, output in zip(examples['instruction'], examples['output']):
            text = qa_prompt.format(instruction, output) + EOS_TOKEN
            texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [10]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 16,
    packing=False,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        max_steps = 2000,
        warmup_steps = 10,
        warmup_ratio = 0.1,
        num_train_epochs = 1,

        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "linear",
        seed = 5609,
        output_dir = "checkpoint_gemma2-2b-it",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [11]:
import gc
torch.cuda.empty_cache()
gc.collect()

83

In [12]:
import os

checkpoint_dir = "../training/checkpoint_gemma2-2b-it/checkpoint-1000/"

if os.path.exists(checkpoint_dir):
    trainer_stats = trainer.train(resume_from_checkpoint=checkpoint_dir)
else:
    print(f"Checkpoint directory {checkpoint_dir} not found, starting fresh training.")
    trainer_stats = trainer.train() 

Unsloth: Setting lr = 1.00e-05 instead of 5.00e-05 for embed_tokens.


/home/baekkwsl/anaconda3/envs/gmlb/lib/python3.10/site-packages/transformers/trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoin

Step,Training Loss
1001,0.666500
1002,1.013700
1003,0.817500
1004,0.838800
1005,1.047600
1006,0.666500
1007,0.776600
1008,0.480900
1009,1.213500
1010,0.833600


AUTOTUNE bmm(16x256x256, 16x256x256)
  triton_bmm_158 0.0175 ms 100.0%
  triton_bmm_162 0.0205 ms 85.5%
  triton_bmm_159 0.0225 ms 77.7%
  triton_bmm_165 0.0245 ms 71.4%
  triton_bmm_161 0.0246 ms 71.2%
  triton_bmm_166 0.0246 ms 71.2%
  triton_bmm_154 0.0266 ms 65.7%
  triton_bmm_153 0.0276 ms 63.3%
  triton_bmm_167 0.0297 ms 58.9%
  triton_bmm_157 0.0299 ms 58.6%
SingleProcess AUTOTUNE benchmarking takes 2.0745 seconds and 0.4624 seconds precompiling
AUTOTUNE bmm(16x256x256, 16x256x256)
  bmm 0.0184 ms 100.0%
  triton_bmm_174 0.0256 ms 72.0%
  triton_bmm_185 0.0266 ms 69.2%
  triton_bmm_184 0.0287 ms 64.3%
  triton_bmm_177 0.0297 ms 62.1%
  triton_bmm_176 0.0307 ms 60.0%
  triton_bmm_181 0.0307 ms 60.0%
  triton_bmm_175 0.0317 ms 58.1%
  triton_bmm_178 0.0317 ms 58.1%
  triton_bmm_186 0.0317 ms 58.1%
SingleProcess AUTOTUNE benchmarking takes 2.0796 seconds and 0.0008 seconds precompiling
AUTOTUNE bmm(16x256x256, 16x256x256)
  bmm 0.0215 ms 100.0%
  triton_bmm_233 0.0276 ms 77.8%
  tr

In [20]:
# 허깅페이스 레포지토리용 파라미터
#base_model = "unsloth/gemma-2-2b-bnb-4bit" # pretrained 모델,
huggingface_repo_gguf = "seungjune/RBCadvisor_GemmaFondo_gguf" # 업로드할 레포지토리

quantization_method = "q4_k_m"   # "q8_0"  "f16" "q8_0" "q4_k_m" "q5_k_m"
# Hub 에 GGUF 업로드
model.push_to_hub_gguf(
    huggingface_repo_gguf,
    tokenizer,
    quantization_method=quantization_method,
    token=hf_token,
)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.59 out of 15.5 RAM for saving.


100%|███████████████████████████████████████████████████████| 26/26 [00:00<00:00, 49.40it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting gemma2 model. Can use fast conversion = False.


...

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/seungjune/RBCadviser_GemmaFondo_gguf
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q4_K_M.gguf:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved GGUF to https://huggingface.co/seungjune/RBCadviser_GemmaFondo_gguf
